In [48]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import joblib
import ast
from tqdm import tqdm
tqdm.pandas()

In [3]:
filepath='../data/all_blocks_0926.csv'
models = joblib.load('../data/models/all_models.jblb')
data = joblib.load('../data/processed/data_1003.jblb')['df']


/Users/bradeneberhard/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
models['fv_model']['features']

['thrower_x',
 'thrower_y',
 'possession_num',
 'possession_throw',
 'game_quarter',
 'quarter_point',
 'score_diff',
 'times']

In [32]:
fv_before_throw_df = models['fv_model']['scaler'].transform(data[models['fv_model']['features']])
fv_before_throw = models['fv_model']['model'].predict_proba(fv_before_throw_df)[:,1]
receiver_features = [x.replace('thrower', 'receiver') for x in models['fv_model']['features']]
fv_after_throw_df = data[receiver_features]
fv_after_throw_df = fv_after_throw_df.rename(columns={'receiver_x': 'thrower_x', 'receiver_y': 'thrower_y'})



fv_opponent_after_throw_df = fv_after_throw_df.copy()
fv_opponent_after_throw_df.loc[:, 'thrower_x'] = -fv_opponent_after_throw_df.loc[:, 'thrower_x']
fv_opponent_after_throw_df.loc[:, 'thrower_y'] = (120 - fv_opponent_after_throw_df.loc[:, 'thrower_y']).clip(lower=20, upper=100)
fv_opponent_after_throw_df.loc[:, 'possession_num'] += 1
fv_opponent_after_throw_df.loc[:, 'possession_throw'] = 1
fv_opponent_after_throw_df.loc[:, 'score_diff'] = -fv_opponent_after_throw_df.loc[:, 'score_diff']

fv_after_throw_df = models['fv_model']['scaler'].transform(fv_after_throw_df)
fv_after_throw = models['fv_model']['model'].predict_proba(fv_after_throw_df)[:,1]

fv_opponent_after_throw_df = models['fv_model']['scaler'].transform(fv_opponent_after_throw_df)
fv_opponent_after_throw = models['fv_model']['model'].predict_proba(fv_opponent_after_throw_df)[:,1]

data['ec'] = np.where(data['turnover'] == 1, 
                      fv_opponent_after_throw - fv_before_throw,  # Use opponent FV if turnover
                      fv_after_throw - fv_before_throw)  # Otherwise use regular FV

In [61]:
thrower_counts = data['thrower'].value_counts()
valid_throwers = thrower_counts[thrower_counts > 200].index
filtered_data = data[data['thrower'].isin(valid_throwers)]
result = filtered_data.groupby(['thrower']).agg({'ec': 'mean'}).sort_values('ec')
result

,ec
thrower,
bgfroerer,-0.003982
ocable,-0.001527
gmartin,-0.001221
mhanna,0.001316
awilsonho,0.001742
...,...
jfloyd,0.038382
bthoennes,0.038457
amuraoka,0.039749


In [49]:
data['current_line'] = data['current_line'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_exploded = data.explode('current_line')


results = []
def get_unique_count(player):
    unique_combos = df_exploded[df_exploded['current_line'] == player].drop_duplicates(
        subset=['gameID', 'quarter_point', 'home_team_score', 'away_team_score', 'possession_num']
    )
    return len(unique_combos)
unique_players = df_exploded['thrower'].unique()
for player in tqdm(unique_players):
    count = get_unique_count(player)
    results.append({'thrower': player, 'unique_count': count})
final_df = pd.DataFrame(results)

100%|██████████| 1560/1560 [02:27<00:00, 10.55it/s]


In [62]:
foo = pd.merge(result, final_df, how='left', on='thrower')
foo['ec_per_possession'] = foo.ec / foo.unique_count

In [63]:
foo.sort_values('ec_per_possession')

,thrower,ec,unique_count,ec_per_possession
0,bgfroerer,-0.003982,286,-0.000014
1,ocable,-0.001527,445,-0.000003
2,gmartin,-0.001221,876,-0.000001
4,awilsonho,0.001742,709,0.000002
3,mhanna,0.001316,385,0.000003
...,...,...,...,...
406,cbrown1,0.035932,171,0.000210
418,sroberts,0.047067,207,0.000227
373,amiller1,0.029097,119,0.000245
416,amuraoka,0.039749,148,0.000269
